In [13]:
import pandas as pd
import numpy as np
import requests
import json
from scripts.vectology_functions import create_aaa_distances, embed_text

In [14]:
# get the EBI UKB data
#get ebi data
url='https://raw.githubusercontent.com/EBISPOT/EFO-UKB-mappings/master/UK_Biobank_master_file.tsv'
ebi_df = pd.read_csv(url,sep='\t')
#create test set
print(ebi_df.head())
print(ebi_df.shape)

                                ZOOMA QUERY       MAPPED_TERM_LABEL  \
0           Vascular disorders of intestine        vascular disease   
1                              Gonarthrosis  osteoarthritis || knee   
2  Psoriatic and enteropathic arthropathies     psoriatic arthritis   
3          Pain associated with micturition                 dysuria   
4                                Other mood           mood disorder   

            MAPPED_TERM_URI MAPPING_TYPE  \
0  EFO_0004264, EFO_0009431        Broad   
1               EFO_0004616        Broad   
2               EFO_0003778      ? Broad   
3               EFO_0003901      ? Broad   
4               EFO_0004247      ? Broad   

  ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE COMMENTS/TICKET   AI  
0                                       K55            DONE  NaN  
1                                       M17            DONE  NaN  
2                                       M07            DONE  NaN  
3                                       

In [15]:
# preprocess
process_text=[]
for k,g in ebi_df.groupby(np.arange(len(ebi_df))//20):
    params={'text_list':list(g['ZOOMA QUERY'])}
    process_res = requests.post('http://vectology-api.mrcieu.ac.uk/preprocess',data=json.dumps(params))
    process_text.extend([d['result'].replace('unspecified','').replace('nec','') for d in process_res.json()])
print(len(process_text))
    
ebi_df.loc[:, 'processed'] = process_text
print(ebi_df.head())

1565
                                ZOOMA QUERY       MAPPED_TERM_LABEL  \
0           Vascular disorders of intestine        vascular disease   
1                              Gonarthrosis  osteoarthritis || knee   
2  Psoriatic and enteropathic arthropathies     psoriatic arthritis   
3          Pain associated with micturition                 dysuria   
4                                Other mood           mood disorder   

            MAPPED_TERM_URI MAPPING_TYPE  \
0  EFO_0004264, EFO_0009431        Broad   
1               EFO_0004616        Broad   
2               EFO_0003778      ? Broad   
3               EFO_0003901      ? Broad   
4               EFO_0004247      ? Broad   

  ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE COMMENTS/TICKET   AI  \
0                                       K55            DONE  NaN   
1                                       M17            DONE  NaN   
2                                       M07            DONE  NaN   
3                              

In [ ]:
# embed all the variables using BioSentVec
def encode_traits(trait_df):

    vectorList=[]

    #loop through 10 rows at a time
    for k,g in trait_df.groupby(np.arange(len(trait_df))//10):
        #get processed text
        textList=list(g['processed'])
        # embed using BioSentVec initially
        res = embed_text(textList,'BioSentVec')
        
        #add vectors to list
        for i in range(0,len(textList)):
            vectorList.append(res[i])

    print(len(vectorList),'vectors created')        
    trait_df['BioSentVec'] = vectorList
    return trait_df

ebi_df = encode_traits(ebi_df)
print(trait_df.head())